In [22]:
import pandas as pd
import numpy as np
import torch
import faiss
import pickle
from sentence_transformers import SentenceTransformer

!pip freeze > requirements.txt

In [4]:
model_save_path = r'C:\Users\Intel7\Desktop\MAS\all-MiniLM-L6-v2'
model = SentenceTransformer(model_save_path)

In [3]:
df = pd.read_csv(r'C:\Users\Intel7\Desktop\MAS\Dataset\data.csv', encoding="ISO-8859-1")

df['OriginalDescription'] = df['Description']

# Data Cleaning
df.dropna(subset=['Description'], inplace=True)  # Remove rows with missing descriptions
df['Description'] = df['Description'].str.lower()  # Convert to lowercase
df['Description'] = df['Description'].str.replace(r'[^a-zA-Z0-9 ]', '', regex=True)  # Remove special characters
# elimination of NaN values
df.dropna(inplace=True)
# elimination of duplicate rows
df.drop_duplicates(inplace=True)
# elimination of cancelled orders
df = df[~df['InvoiceNo'].str.startswith('C')]

# remove duplicate descriptions 
df.drop_duplicates(subset=['Description'], inplace=True, ignore_index=True)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,OriginalDescription
0,536365,85123A,white hanging heart tlight holder,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER
1,536365,71053,white metal lantern,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,WHITE METAL LANTERN
2,536365,84406B,cream cupid hearts coat hanger,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,CREAM CUPID HEARTS COAT HANGER
3,536365,84029G,knitted union flag hot water bottle,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,KNITTED UNION FLAG HOT WATER BOTTLE
4,536365,84029E,red woolly hottie white heart,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.


In [30]:
# Encode the product descriptions
descriptions = df['Description'].tolist()
encoded_descriptions = model.encode(descriptions)

# Create a Faiss index
dim = encoded_descriptions.shape[1]
index = faiss.IndexFlatL2(dim)

# Add the encoded descriptions to the Faiss index
index.add(encoded_descriptions)

In [31]:
def search_faiss(query, model, index, k=5):
    query = model.encode([query])[0]
    query = np.expand_dims(query, axis=0)
    Distances, Indices = index.search(query, k)
    return Distances, Indices

# Search for similar products
query = "big mug"
Distances, Indices = search_faiss(query, model, index)

# Print the similar products
for i in range(len(Indices[0])):
    print(Indices[0][i])
    print(df['OriginalDescription'].iloc[Indices[0][i]])
    print('Distance:', Distances[0][i])
    print()

2871
BIG POLKADOT MUG
Distance: 0.45231256

1065
GLAMOROUS  MUG
Distance: 0.52221566

1478
POTTERING MUG
Distance: 0.6258617

925
HOME SWEET HOME MUG
Distance: 0.6635146

1160
LOCAL CAFE MUG
Distance: 0.6828939



In [32]:
from agno.agent import Agent

class SearchAgent(Agent):
    def __init__(self, index, model, df):
        super().__init__()
        self.index = index
        self.model = model
        self.df = df

    def search(self, query, k=5):
        encoded_query = self.model.encode([query])
        distances, indices = self.index.search(encoded_query, k)
        results = [self.df['OriginalDescription'].iloc[indices[0][i]] for i in range(len(indices[0]))]
        return results

# Initialize the search agent
search_agent = SearchAgent(index=index, model=model, df=df)

# Example usage
query = "Phone"
results = search_agent.search(query)
print("Search results:")
for result in results:
    print(result)

Search results:
CRYSTAL FROG PHONE CHARM
CRYSTAL STILETTO PHONE CHARM
CRYSTAL SEA HORSE PHONE CHARM
PINK CRYSTAL SKULL PHONE CHARM
PINK CRYSTAL GUITAR PHONE CHARM


## Agents

In [10]:
from agno.agent import Agent, RunResponse
from agno.models.ollama import Ollama
from agno.tools.duckduckgo import DuckDuckGoTools


agent = Agent(
    model=Ollama(id="llama3.2"),
    markdown=False,
    description="You are chatting agent chat can answer questions and ask follow-up questions.",
)

response = agent.run("Whats happening in France?")
print(response.content)

France is a vibrant country with a rich history, culture, and politics. Here's an overview of what's currently happening:

1. **Protests and strikes**: As in many European countries, France has seen ongoing protests and strikes, particularly among pensioners, students, and workers. The main issues include economic inequality, pension reform, and government policies.
2. **Climate change and environmental policies**: France is taking steps to address climate change and reduce carbon emissions. In 2020, the country set ambitious targets to become carbon neutral by 2050. However, critics argue that these goals are not being met quickly enough.
3. **Brexit and EU politics**: As a key player in European politics, France has been impacted by Brexit. The UK's departure from the EU has created uncertainty for French businesses and citizens alike. France is also playing a significant role in shaping EU policies on issues like migration and security.
4. **French presidential elections**: In 2022,

In [21]:
from agno.agent import Agent, RunResponse
from agno.models.huggingface import HuggingFace
from agno.tools.googlesearch import GoogleSearchTools
from agno.tools.wikipedia import WikipediaTools
from agno.tools.yfinance import YFinanceTools
import os
from agno.models.ollama import Ollama
from agno.tools.duckduckgo import DuckDuckGoTools
from pydantic import BaseModel, Field

class Product(BaseModel):
    product_name: str = Field(..., title="Product Name")
    description: str = Field(..., title="Description")
    url: str = Field(..., title="URL")

class WebSearchResults(BaseModel):
    products: list[Product] = Field(..., title="Products")

web_agent = Agent(
    model=Ollama(id = "llama3.2"),
    tools=[DuckDuckGoTools(), GoogleSearchTools()],
    show_tool_calls=True,
    description="You are an web search agent to help user for product recommendation.",
    # response_model=WebSearchResults,
    instructions=[
        "You will be provides with an query from user about which type of product he/she wants to buy.",
        "Carefully read the query and provide the user with the top 5 products that match the query.",
        "Make sure to provide the product name, description and URL of the product.",
    ],
    markdown=True,
    stream=True,
)

web_agent.print_response(
    "Phone", stream=True
)

c:\Users\Intel7\Desktop\MAS\myenv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter
support
  warnings.warn('install "ipywidgets" for Jupyter support')

In [39]:
from agno.workflow import Workflow

class WebSearchResults(BaseModel):
    products: list[str]

class SearchWorkflow(Workflow):
    
    # Define chat agent
    chat_agent: Agent = Agent(
        model=Ollama(id="llama3.2"),
        markdown=False,
        description="You have to help the user for product recommendation.",
        instructions=[
            "You will be provided with a 5 product recommendations based from web search agent.",
            "You have to present these recommendations to the user and summarize the recommendations.",
            "You have to ask the user appropriate questions to understand the user's requirement.",
        ],
    )

    # Define web search agent
    web_agent: Agent = Agent(
            model=Ollama(id = "llama3.2"),
            tools=[GoogleSearchTools(fixed_max_results=100000), DuckDuckGoTools(fixed_max_results=100000)],
            show_tool_calls=True,
            description="You are an web search agent to help user for product recommendation.",
            response_model=WebSearchResults,
            instructions=[
                "You will be provides with an query from user about which type of product he/she wants to buy.",
                "Carefully read the query and provide the user with the top 5 products that match the query.",
                "Make sure to provide the product name, description and URL of the product.",
                "Give it as a list of products.",
            ],
            structured_outputs=False,
            markdown=True,
            stream=True,
        )

    # Define workflow steps
    def run(self, query: str = "") -> RunResponse:
        
        # Get web search response
        web_response = self.web_agent.run(query)
        web_content = web_response.content

        # Get chat response
        chat_response = self.chat_agent.run(web_content)
        chat_content = chat_response.content

        # Combine chat and web search responses
        content = f"{chat_content}\n\n{web_content}"
        return RunResponse(content=content)
    
# Initialize the workflow
workflow = SearchWorkflow()

# Run the workflow
response = workflow.run(query="Phone")

WARNING  Could not run function duckduckgo_search(max_results=5, query=phone products)

ERROR    https://html.duckduckgo.com/html 202 Ratelimit                                                            
         Traceback (most recent call last):                                                                        
           File "c:\Users\Intel7\Desktop\MAS\myenv\Lib\site-packages\agno\tools\function.py", line 359, in execute 
             self.result = self.function.entrypoint(**entrypoint_args, **self.arguments)                           
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                           
           File "c:\Users\Intel7\Desktop\MAS\myenv\Lib\site-packages\pydantic\_internal\_validate_call.py", line   
         38, in wrapper_function                                                                                   
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "c:\Users\Intel7\Desktop\MAS\myenv\Lib\site-packages\pydantic\_internal\_validate_call.py", line   
         111, in __call__                                                                                          
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "c:\Users\Intel7\Desktop\MAS\myenv\Lib\site-packages\agno\tools\duckduckgo.py", line 68, in        
         duckduckgo_search                                                                                         
             return json.dumps(ddgs.text(keywords=query, max_results=(self.fixed_max_results or max_results)),     
         indent=2)                                                                                                 
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^      
           File "c:\Users\Intel7\Desktop\MAS\myenv\Lib\site-packages\duckduckgo_search\duckduckgo_search.py", line 
         252, in text                                                                                              
             raise DuckDuckGoSearchException(err)                                                                  
         duckduckgo_search.exceptions.DuckDuckGoSearchException: https://html.duckduckgo.com/html 202 Ratelimit

WARNING  Failed to convert response to response_model